In [1]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21+0.gcf6eeacde.dirty


In [2]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n28", 5555)

connected to arkouda server tcp://*:5555


In [3]:
n = 100
m = 1000
s = 2
x = 2
y = 2

In [4]:
import time

### Build graph from randomly generated source and destination arrays.
# 1. Use Arkouda's randint to generate the random edge arrays.
src = ak.randint(0, n, m, seed=s*2)
dst = ak.randint(0, n, m, seed=s*4)

# 2. Build temporary property graph to get sorted edges and nodes lists.
temp_prop_graph = ar.PropGraph()
start = time.time()
temp_prop_graph.add_edges_from(src, dst)
end = time.time()
build_time = end - start
print(f"Building property graph with {len(temp_prop_graph)} vertices and "
        f"{temp_prop_graph.size()} "
        f"edges took {round(build_time,2)} seconds.")

### Generate node labels and edge relationships for the graph.
# 1. Extract node and edge information.
num_edges = temp_prop_graph.size()
num_nodes = len(temp_prop_graph)
edges = temp_prop_graph.edges()
nodes = temp_prop_graph.nodes()

# 2. Generate random integer attributes for both vertices and edges.
int_array_nodes = ak.randint(-1, x, num_nodes, dtype=ak.dtype('int64'))
uint_array_nodes = ak.randint(0, x, num_nodes, dtype=ak.dtype('uint64'))
real_array_nodes = ak.randint(0, x, num_nodes, dtype=ak.dtype('float64'))
bool_array_nodes = ak.randint(0, x, num_nodes, dtype=ak.dtype('bool'))
strings_array_nodes = ak.random_strings_uniform(minlen=2, maxlen=3, size=num_nodes, characters="uppercase")
categorical_array_nodes = ak.Categorical(strings_array_nodes)

int_array_edges = ak.randint(-1, x, num_edges, dtype=ak.dtype('int64'))
uint_array_edges = ak.randint(0, x, num_edges, dtype=ak.dtype('uint64'))
real_array_edges = ak.randint(0, x, num_edges, dtype=ak.dtype('float64'))
bool_array_edges = ak.randint(0, x, num_edges, dtype=ak.dtype('bool'))
strings_array_edges = ak.random_strings_uniform(minlen=2, maxlen=3, size=num_edges, characters="uppercase")
categorical_array_edges = ak.Categorical(strings_array_edges)

# 3. Create dataframe to load into a new property graph.
edge_df = ak.DataFrame({"src":edges[0], "dst":edges[1], 
                        "ints":int_array_edges, 
                        "uints":uint_array_edges,
                        "reals":real_array_edges,
                        "bools":bool_array_edges,
                        "strings":strings_array_edges,
                        "categoricals":categorical_array_edges})
node_df = ak.DataFrame({"nodes":nodes, 
                        "ints":int_array_nodes, 
                        "uints":uint_array_nodes,
                        "reals":real_array_nodes,
                        "bools":bool_array_nodes,
                        "strings":strings_array_nodes,
                        "categoricals":categorical_array_nodes})

# 4. Create new property graph with node labels and edge relationships.
prop_graph = ar.PropGraph()
prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
prop_graph.load_node_attributes(node_df, node_column="nodes")

### Create the subgraph we are searching for.
# 1. Create labels and relationships to search for.
src_list = [0, 1, 2, 2, 3]
dst_list = [1, 2, 0, 3, 0]
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

subgraph_int_array_nodes = ak.array([-1] * len(subgraph_nodes))
subgraph_uint_array_nodes = ak.array([0] * len(subgraph_nodes), dtype=ak.dtype('uint64'))
subgraph_real_array_nodes = ak.array([0.0] * len(subgraph_nodes))
subgraph_bool_array_nodes = ak.array([True] * len(subgraph_nodes))
subgraph_strings_array_nodes = ak.array(["AB"] * len(subgraph_nodes))
subgraph_categorical_array_nodes = ak.Categorical(subgraph_strings_array_nodes)

subgraph_int_array_edges = ak.array([-1] * len(src_list))
subgraph_uint_array_edges = ak.array([0] * len(src_list), dtype=ak.dtype('uint64'))
subgraph_real_array_edges = ak.array([0.0] * len(src_list))
subgraph_bool_array_edges = ak.array([True] * len(src_list))
subgraph_strings_array_edges = ak.array(["AB"] * len(src_list))
subgraph_categorical_array_edges = ak.Categorical(subgraph_strings_array_edges)

Building property graph with 100 vertices and 949 edges took 3.03 seconds.


In [5]:
"""TEST COMBINED PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                          "ints":subgraph_int_array_edges, 
                          "uints":subgraph_uint_array_edges,
                          "reals":subgraph_real_array_edges,
                          "bools":subgraph_bool_array_edges,
                          "strings":subgraph_strings_array_edges,
                          "categoricals":subgraph_categorical_array_edges})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          "ints":subgraph_int_array_nodes, 
                          "uints":subgraph_uint_array_nodes,
                          "reals":subgraph_real_array_nodes,
                          "bools":subgraph_bool_array_nodes,
                          "strings":subgraph_strings_array_nodes,
                          "categoricals":subgraph_categorical_array_nodes})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column = "nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 116.0 isos inside of the graph


In [6]:
"""TEST VERTEX PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          "ints":subgraph_int_array_nodes, 
                          "uints":subgraph_uint_array_nodes,
                          "reals":subgraph_real_array_nodes,
                          "bools":subgraph_bool_array_nodes,
                          "strings":subgraph_strings_array_nodes,
                          "categoricals":subgraph_categorical_array_nodes})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column = "nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 304.0 isos inside of the graph


In [7]:
"""TEST EDGE PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                          "ints":subgraph_int_array_edges, 
                          "uints":subgraph_uint_array_edges,
                          "reals":subgraph_real_array_edges,
                          "bools":subgraph_bool_array_edges,
                          "strings":subgraph_strings_array_edges,
                          "categoricals":subgraph_categorical_array_edges})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 303.0 isos inside of the graph


In [8]:
"""TEST NO PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 670.0 isos inside of the graph


In [9]:
"""VF2-PS COMBINED GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                          "ints":subgraph_int_array_edges, 
                          "uints":subgraph_uint_array_edges,
                          "reals":subgraph_real_array_edges,
                          "bools":subgraph_bool_array_edges,
                          "strings":subgraph_strings_array_edges,
                          "categoricals":subgraph_categorical_array_edges})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          "ints":subgraph_int_array_nodes, 
                          "uints":subgraph_uint_array_nodes,
                          "reals":subgraph_real_array_nodes,
                          "bools":subgraph_bool_array_nodes,
                          "strings":subgraph_strings_array_nodes,
                          "categoricals":subgraph_categorical_array_nodes})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column = "nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 116.0 isos inside of the graph


In [10]:
"""VF2-PS VERTEX PICKER GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          "ints":subgraph_int_array_nodes, 
                          "uints":subgraph_uint_array_nodes,
                          "reals":subgraph_real_array_nodes,
                          "bools":subgraph_bool_array_nodes,
                          "strings":subgraph_strings_array_nodes,
                          "categoricals":subgraph_categorical_array_nodes})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column = "nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 304.0 isos inside of the graph


In [11]:
"""VF2-PS EDGE PICKER GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                          "ints":subgraph_int_array_edges, 
                          "uints":subgraph_uint_array_edges,
                          "reals":subgraph_real_array_edges,
                          "bools":subgraph_bool_array_edges,
                          "strings":subgraph_strings_array_edges,
                          "categoricals":subgraph_categorical_array_edges})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 303.0 isos inside of the graph


In [12]:
"""VF2-PS NO ATTRIBUTES GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices")
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 670.0 isos inside of the graph
